In [1]:
from xno.data.datasets.hdf5_dataset import H5pyDataset
from utils import MatReader
from torch.utils.data import DataLoader, TensorDataset, Dataset, default_collate
import numpy as np

In [2]:
import torch
import matplotlib.pyplot as plt
import sys
from xno.models import XNO
from xno.utils import count_model_params
from xno.training import AdamW
from xno.training.incremental import IncrementalFNOTrainer
from xno.data.transforms.data_processors import IncrementalDataProcessor
from xno import LpLoss, H1Loss

In [3]:
data_path="data/train_IC2.npz"

In [4]:
""" Model configurations """

ntrain = 900
ntest = 100

batch_size = 20
learning_rate = 0.001

epochs = 500
step_size = 50   # weight-decay step size
gamma = 0.5      # weight-decay rate

wavelet = 'db6'  # wavelet basis function
level = 3        # lavel of wavelet decomposition
width = 96       # uplifting dimension
layers = 4       # no of wavelet layers

h = 40           # total grid size divided by the subsampling rate
grid_range = 1
in_channel = 2   # (a(x), x) for this case

In [5]:
# Data is of the shape (number of samples, grid size)
data = np.load(data_path)
x, t, u_train = data["x"], data["t"], data["u"]  # N x nt x nx

x_data = u_train[:, 0, :]  # N x nx, initial solution
y_data = u_train[:, -2, :]  # N x nx, final solution

x_data = torch.tensor(x_data)
y_data = torch.tensor(y_data)

x_train = x_data[:ntrain,:]
y_train = y_data[:ntrain,:]
x_test = x_data[-ntest:,:]
y_test = y_data[-ntest:,:]

x_train = x_train[:, :, None]
x_test = x_test[:, :, None]


In [6]:
# Define the custom Dataset
class DictDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return {'x': self.x[idx], 'y': self.y[idx]}

In [7]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

(torch.Size([900, 40, 1]),
 torch.Size([900, 40]),
 torch.Size([100, 40, 1]),
 torch.Size([100, 40]))

In [8]:
x_train = x_train.permute(0, 2, 1)
y_train = y_train.unsqueeze(1)
x_test = x_test.permute(0, 2, 1)
y_test = y_test.unsqueeze(1)

In [9]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

(torch.Size([900, 1, 40]),
 torch.Size([900, 1, 40]),
 torch.Size([100, 1, 40]),
 torch.Size([100, 1, 40]))

In [10]:
train_loader = DictDataset(x_train, y_train)
test_loader = DictDataset(x_test, y_test)

In [11]:
train_loader = DataLoader(train_loader, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_loader, batch_size=batch_size, shuffle=True)
test_loader = {
    40: test_loader
}

In [12]:
batch = next(iter(train_loader))
type(train_loader), type(batch), batch['x'].shape, batch['y'].shape

(torch.utils.data.dataloader.DataLoader,
 dict,
 torch.Size([20, 1, 40]),
 torch.Size([20, 1, 40]))

In [13]:
batch = next(iter(test_loader[40]))
type(test_loader), type(batch), batch['x'].shape, batch['y'].shape

(dict, dict, torch.Size([20, 1, 40]), torch.Size([20, 1, 40]))

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
model = XNO(
    max_n_modes=(16, ),
    n_modes=(16, ),
    hidden_channels=width,
    in_channels=1,
    out_channels=1,
    transformation="lno",
    # transformation_kwargs={"wavelet_level": level, "wavelet_size": [40], "wavelet_filter":[wavelet]}, 
    n_layers=layers
)
model = model.to(device)
n_params = count_model_params(model)

======== Selected Kernel Description =======
Dimentionality: 1D
Transformation: [ Laplace Neural Operator (LNO) Kernel ]
>>> Overview:
The LNO uses a pole-residue formulation to compute solutions to PDEs in the Laplace domain.
This kernel is highly effective for problems requiring stability and steady-state solutions.

>>> Key Features:
- Specially designed for systems dominated by Laplacian dynamics.
- Balances transient and steady-state components.

>>> Reference:
Cao, Q. et al. 'LNO: Laplace Neural Operator for Solving Differential Equations'.
Link: https://arxiv.org/pdf/2303.10528

================== Config ==================
>>> Normaliztion: group_norm
>>> Activation Function: 



In [16]:
optimizer = AdamW(model.parameters(), lr=8e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

In [17]:
data_transform = IncrementalDataProcessor(
    in_normalizer=None,
    out_normalizer=None,
    device=device,
    subsampling_rates=[2, 1],
    dataset_resolution=2048,
    dataset_indices=[2],
    epoch_gap=10,
    verbose=True,
)

data_transform = data_transform.to(device)

Original Incre Res: change index to 0
Original Incre Res: change sub to 2
Original Incre Res: change res to 1024


In [18]:
l2loss = LpLoss(d=2, p=2)
h1loss = H1Loss(d=2)
train_loss = h1loss
eval_losses = {"h1": h1loss, "l2": l2loss}
print("\n### N PARAMS ###\n", n_params)
print("\n### OPTIMIZER ###\n", optimizer)
print("\n### SCHEDULER ###\n", scheduler)
print("\n### LOSSES ###")
print("\n### INCREMENTAL RESOLUTION + GRADIENT EXPLAINED ###")
print(f"\n * Train: {train_loss}")
print(f"\n * Test: {eval_losses}")
sys.stdout.flush()


### N PARAMS ###
 2473441

### OPTIMIZER ###
 AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 0.008
    lr: 0.008
    weight_decay: 0.0001
)

### SCHEDULER ###

### LOSSES ###

### INCREMENTAL RESOLUTION + GRADIENT EXPLAINED ###

 * Train: <xno.losses.data_losses.H1Loss object at 0x149d528f0>

 * Test: {'h1': <xno.losses.data_losses.H1Loss object at 0x149d528f0>, 'l2': <xno.losses.data_losses.LpLoss object at 0x14bc96260>}


In [19]:
# Finally pass all of these to the Trainer
trainer = IncrementalFNOTrainer(
    model=model,
    n_epochs=10,
    data_processor=data_transform,
    device=device,
    verbose=True,
    incremental_loss_gap=False,
    incremental_grad=True,
    incremental_grad_eps=0.9999,
    incremental_loss_eps = 0.001,
    incremental_buffer=5,
    incremental_max_iter=1,
    incremental_grad_max_iter=2,
)

In [20]:
trainer.train(
    train_loader,
    test_loader,
    optimizer,
    scheduler,
    regularizer=False,
    training_loss=train_loss,
    eval_losses=eval_losses,
    save_every=5, 
    save_dir="save/lorenz1d_lno"
)

Training on 900 samples
Testing on [100] samples         on resolutions [40].
Raw outputs of shape torch.Size([20, 1, 20])
[0] time=16.54, avg_loss=0.7486, train_err=14.9711
Eval: 40_h1=0.9052, 40_l2=0.4813
[Rank 0]: saved training state to save/lorenz1d_lno
[1] time=16.19, avg_loss=0.5854, train_err=11.7079
Eval: 40_h1=0.8430, 40_l2=0.4560
[2] time=19.20, avg_loss=0.5757, train_err=11.5131
Eval: 40_h1=0.7817, 40_l2=0.4850
[3] time=16.47, avg_loss=0.5460, train_err=10.9193
Eval: 40_h1=0.7836, 40_l2=0.4706
[4] time=16.88, avg_loss=0.4959, train_err=9.9184
Eval: 40_h1=0.7021, 40_l2=0.3886
[5] time=18.98, avg_loss=0.4734, train_err=9.4670
Eval: 40_h1=0.6979, 40_l2=0.3936
[Rank 0]: saved training state to save/lorenz1d_lno
[6] time=17.84, avg_loss=0.4804, train_err=9.6081
Eval: 40_h1=0.7262, 40_l2=0.4276
[7] time=17.31, avg_loss=0.4577, train_err=9.1535
Eval: 40_h1=0.7477, 40_l2=0.4021
[8] time=17.16, avg_loss=0.4810, train_err=9.6193
Eval: 40_h1=0.7565, 40_l2=0.4356
[9] time=17.18, avg_lo

{'train_err': 9.366039276123047,
 'avg_loss': 0.46830196380615235,
 'avg_lasso_loss': None,
 'epoch_train_time': 17.177791417001572,
 '40_h1': tensor(0.7125),
 '40_l2': tensor(0.4029)}

In [21]:
# FNO
{'train_err': 6.474369451734755,
 'avg_loss': 0.32371847258673775,
 'avg_lasso_loss': None,
 'epoch_train_time': 2.864431916998001,
 '40_h1': tensor(0.5186),
 '40_l2': tensor(0.2436)}

NameError: name 'tensor' is not defined

In [22]:
# HNO


In [ ]:
# WNO
{'train_err': 6.527459494272867,
 'avg_loss': 0.3263729747136434,
 'avg_lasso_loss': None,
 'epoch_train_time': 14.213203417006298,
 '40_h1': tensor(1.2256),
 '40_l2': tensor(0.8130)}

In [24]:
# LNO
